In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from scipy.sparse import *
import pickle
from sklearn.decomposition import NMF
import multiprocessing

In [2]:
data_path = '../data/'
train = pd.read_csv(data_path + 'train_fill.csv')
test = pd.read_csv(data_path + 'test_fill.csv')
all_data = pd.concat([train, test])

In [3]:
all_data.msno = pd.Categorical(all_data.msno)
all_data.msno = all_data.msno.cat.codes
all_data.song_id = pd.Categorical(all_data.song_id)
all_data.song_id = all_data.song_id.cat.codes

num_users = len(all_data.msno.unique()) 
num_songs = len(all_data.song_id.unique())

pos_X = all_data[all_data.target == 1]
neg_X = all_data[all_data.target == 0]
row1, row2 = pos_X.msno, neg_X.msno
col1, col2 = pos_X.song_id, neg_X.song_id

In [14]:
def get_X(all_data):    
#     all_data.msno = pd.Categorical(all_data.msno)
#     all_data.msno = all_data.msno.cat.codes
#     all_data.song_id = pd.Categorical(all_data.song_id)
#     all_data.song_id = all_data.song_id.cat.codes

#     num_users = len(all_data.msno.unique()) 
#     num_songs = len(all_data.song_id.unique())

#     pos_X = all_data[all_data.target == 1]
#     neg_X = all_data[all_data.target == 0]
#     row1, row2 = pos_X.msno, neg_X.msno
#     col1, col2 = pos_X.song_id, neg_X.song_id
#     print max(row1), max(row2), max(col1), max(col2)
    content1, content2 = np.ones(len(pos_X)), np.ones(len(neg_X)) * -1
    content, row, col = np.concatenate((content1, content2), axis = 0)\
                , np.concatenate((row1,row2), axis = 0), np.concatenate((col1, col2), axis = 0)
    print len(row)
    return csc_matrix((content, (row, col)), \
                   shape=(num_users, num_songs)).toarray()

In [5]:
def get_W():    
    row = all_data.msno
    col = all_data.song_id
    content = np.ones(len(all_data))
    return csc_matrix((content, (row, col)), \
                   shape=(num_users, num_songs)).toarray()

In [16]:
#X = get_X(all_data)
#280118
print len(train)

280119


## x_hat = alpha + beta_u + beta_i + P_u' * Q_i

In [21]:
K = 10
bias = 0.5
alpha = 0.0001
lam = 0.01
beta_u = np.mean(X, axis = 1)
beta_item = np.mean(X, axis = 0)
P = np.random.rand(num_users, K)
Q = np.random.rand(num_songs, K)
pos_length, neg_length = int(len(row1) * 0.8), int(len(row2) * 0.8)
train_row1, train_col1 = row1[:pos_length], col1[:pos_length]
train_row2, train_col2 = row1[:neg_length], col1[:neg_length]
valid_row1, valid_col1 = row1[pos_length:], col1[pos_length:]
valid_row2, valid_col2 = row1[neg_length:], col1[neg_length:]

In [26]:
pool = multiprocessing.Pool(processes=4)
def get_error(row1, col1, row2, col2):
    error = 0
    for i, j in zip(row1, col1):
        y_hat = bias + beta_u[i] + beta_item[j] + np.dot(P[i], Q[j])
        err = 1 - y_hat
        P[i] += alpha * (2 * err * Q[j]  - lam * P[i])
        Q[j] += alpha * (2 * err * P[i]  - lam * Q[j])
        error += err
    for i, j in zip(row2, col2):
        y_hat = bias + beta_u[i] + beta_item[j] + np.dot(P[i], Q[j])
        error += 0 - y_hat   
        P[i] += alpha * (2 * err * Q[j]  - lam * P[i])
        Q[j] += alpha * (2 * err * P[i]  - lam * Q[j])
        error += err
    return error / (len(row1) + len(row2))
def get_train_error():
    return get_error(train_row1, train_col1, train_row2, train_col2)
def get_valid_error():
    return get_error(valid_row1, valid_col1, valid_row2, valid_col2)

In [27]:
for i in range(100):
    print get_train_error()

-2.53796773492
-2.38051826524
-2.24904496382
-2.13677249011
-2.03919147944
-1.95316821852
-1.87644834133
-1.80736284301
-1.74464555017
-1.68731541634
-1.63459818444
-1.58587281851
-1.54063399368
-1.49846527063
-1.45901954337
-1.42200454135
-1.38717190965
-1.35430886565
-1.32323174033
-1.29378091787
-1.2658168269
-1.23921673223
-1.21387214317
-1.18968670154
-1.16657444687
-1.14445838072
-1.12326927033
-1.10294464522
-1.08342795068
-1.06466782943
-1.04661750906
-1.02923427684
-1.01247902762
-0.996315872686
-0.980711800213
-0.965636379156
-0.951061500206
-0.936961148318
-0.92331120235
-0.910089257974
-0.897274470748
-0.884847416603
-0.872789967538
-0.861085180532
-0.849717198021
-0.838671158593
-0.827933116572
-0.817489969526
-0.807329392771
-0.797439780039
-0.78781018967
-0.77843029571
-0.769290343385
-0.760381108496
-0.75169386033
-0.743220327724
-0.734952667971
-0.726883438295
-0.719005569617
-0.711312342457
-0.703797364679
-0.696454551002
-0.689278104054
-0.682262496849
-0.67540245657

In [28]:
print get_valid_error()

-1.87962387758
